# Laboratory №2
---
## Multilayer Perceptrons. Deep learning Computation

### student: Karim Safiullin
### group: 4167

Task: 
1. Setup the environment and run all the notebooks  
2. On previously found dataset create multilayer perceptron network for prediction the value by using high-level APIs and perform the following tasks:  
    1. Compare different activations functions (ReLu, Sigmoid, etc.). See how change the result and try to analyse it. 
    1. Try to change size of training/validation data. Plot how change the result of loss function. Analyse from under/overfitting problems. 
    1.  Use regularization. Analyse the changing of the result. 
    1. Perform the dropout method. Analyse the result. 
    1. Perform the Xavier initialization.


In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
tf.__version__

'2.3.0'

## Reading Dataset  
**Dataset**: Melbourne Housing Market  
**Source**: Kaggle  
**Description**: Melbourne housing clearance data from Jan 2016  

In [3]:
DATASET_PATH = "./data/melb_data.csv"

In [4]:
melbourne_data = pd.read_csv(DATASET_PATH) 

## Column's desciption:
- `Suburb` - name of the suburb **will be used**
- `Adress` - houses location **will not be used**
- `Rooms` - number of rooms in a house **will be used**
- `Type` -  br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential. **will be used**
- `Price` - output column for prediction (price of a house) **will be used**
- `Method` - S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available. **will not be used**
- `SellerG` - name of Real Estate Agent **will not be used**
- `Date` - Date sold **will not be used**
- `Distance` - Distance from CBD (Melbourne city centre) **will be used**
- `Postcode` - postcode of a house same as suburb **will be used**
- `Bedroom2` -  number of bedrooms **will  be used**
- `Bathroom` - amount of bathrooms **will be used**
- `Car` - Number of carspots **will be used**
- `Landsize` -  Land Size **will be used**
- `BuildingArea` - Building Size  **will be used**
- `YearBuilt` - year of a build **will be used**
- `CouncilArea` -  Governing council for the area **will not be used**
- `Lattitude` - latitude of a house **will not be used**
- `Longtitude` - longtitude of a house **will not be used**
- `Regionname` - name of houses region **will be used**
- `Propertycount` -   Number of properties that exist in the suburb. **will be used**

### all strings will be changed at the numbers (for example: `method` column: S - 1, SP - 2 and etc )

In [5]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [6]:
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [7]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [8]:
melbourne_data.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [9]:
type(melbourne_data)

pandas.core.frame.DataFrame

In [123]:
num_words=melbourne_data['Suburb'].unique().size

In [124]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(melbourne_data['Suburb'])
name1_tokenized = tokenizer.texts_to_sequences(melbourne_data['Suburb'])
name1_tokenized_pad = tf.keras.preprocessing.sequence.pad_sequences(name1_tokenized, maxlen=1, truncating='pre')

In [125]:
name1_tokenized_pad

array([[88],
       [88],
       [88],
       ...,
       [49],
       [49],
       [32]], dtype=int32)

In [126]:
melbourne_data=melbourne_data.assign(Suburb=pd.DataFrame(name1_tokenized_pad, columns=['Suburb']))

In [127]:
num_words=melbourne_data['Regionname'].unique().size

In [128]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(melbourne_data['Regionname'])
name1_tokenized = tokenizer.texts_to_sequences(melbourne_data['Regionname'])
name1_tokenized_pad = tf.keras.preprocessing.sequence.pad_sequences(name1_tokenized, maxlen=1, truncating='pre')

In [129]:
melbourne_data=melbourne_data.assign(Regionname=pd.DataFrame(name1_tokenized_pad, columns=['Regionname']))

In [130]:
melbourne_data = melbourne_data.sample(frac = 1) 

In [131]:
melbourne_data = melbourne_data.dropna(axis=0,subset=['Price'])

In [132]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
11655,194,9a Ivan Av,2,u,724000.0,S,O'Brien,22/07/2017,27.0,3196.0,...,1.0,2.0,224.0,104.0,2000.0,Kingston,-38.03710,145.11717,1,2546.0
2525,38,127 George St,4,h,1780000.0,S,Peter,7/11/2016,1.6,3065.0,...,1.0,1.0,186.0,NaN,1902.0,Yarra,-37.80480,144.98060,1,5825.0
11535,80,1/34 Longmuir Rd,2,u,655000.0,S,Darren,15/07/2017,14.5,3087.0,...,1.0,1.0,298.0,98.0,1975.0,Banyule,-37.71098,145.08926,1,2329.0
9832,152,3/13 Graham Rd,3,u,590000.0,S,Mitchell,24/06/2017,31.2,3197.0,...,1.0,1.0,179.0,NaN,NaN,Kingston,-38.07483,145.13133,1,1989.0
11080,254,40 Anzac Av,5,h,790000.0,S,Mason,12/08/2017,26.1,3099.0,...,2.0,2.0,1034.0,225.0,1985.0,Nillumbik,-37.63826,145.19910,7,1345.0


In [133]:
melbourne_data.dtypes

Suburb             int32
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname         int32
Propertycount    float64
dtype: object

In [134]:
melbourne_data.Type

11655    u
2525     h
11535    u
9832     u
11080    h
        ..
10480    t
6668     u
9390     u
12868    h
6221     h
Name: Type, Length: 13580, dtype: object

In [135]:
melbourne_data.Type.unique()

array(['u', 'h', 't'], dtype=object)

In [136]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
11655,194,9a Ivan Av,2,u,724000.0,S,O'Brien,22/07/2017,27.0,3196.0,...,1.0,2.0,224.0,104.0,2000.0,Kingston,-38.03710,145.11717,1,2546.0
2525,38,127 George St,4,h,1780000.0,S,Peter,7/11/2016,1.6,3065.0,...,1.0,1.0,186.0,NaN,1902.0,Yarra,-37.80480,144.98060,1,5825.0
11535,80,1/34 Longmuir Rd,2,u,655000.0,S,Darren,15/07/2017,14.5,3087.0,...,1.0,1.0,298.0,98.0,1975.0,Banyule,-37.71098,145.08926,1,2329.0
9832,152,3/13 Graham Rd,3,u,590000.0,S,Mitchell,24/06/2017,31.2,3197.0,...,1.0,1.0,179.0,NaN,NaN,Kingston,-38.07483,145.13133,1,1989.0
11080,254,40 Anzac Av,5,h,790000.0,S,Mason,12/08/2017,26.1,3099.0,...,2.0,2.0,1034.0,225.0,1985.0,Nillumbik,-37.63826,145.19910,7,1345.0


In [137]:
melbourne_data=melbourne_data.fillna(-1)
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
11655,194,9a Ivan Av,2,u,724000.0,S,O'Brien,22/07/2017,27.0,3196.0,...,1.0,2.0,224.0,104.0,2000.0,Kingston,-38.03710,145.11717,1,2546.0
2525,38,127 George St,4,h,1780000.0,S,Peter,7/11/2016,1.6,3065.0,...,1.0,1.0,186.0,-1.0,1902.0,Yarra,-37.80480,144.98060,1,5825.0
11535,80,1/34 Longmuir Rd,2,u,655000.0,S,Darren,15/07/2017,14.5,3087.0,...,1.0,1.0,298.0,98.0,1975.0,Banyule,-37.71098,145.08926,1,2329.0
9832,152,3/13 Graham Rd,3,u,590000.0,S,Mitchell,24/06/2017,31.2,3197.0,...,1.0,1.0,179.0,-1.0,-1.0,Kingston,-38.07483,145.13133,1,1989.0
11080,254,40 Anzac Av,5,h,790000.0,S,Mason,12/08/2017,26.1,3099.0,...,2.0,2.0,1034.0,225.0,1985.0,Nillumbik,-37.63826,145.19910,7,1345.0


In [138]:
type_col = melbourne_data["Type"].map({'h':0,'t':1,'u':2})

In [139]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
11655,194,9a Ivan Av,2,u,724000.0,S,O'Brien,22/07/2017,27.0,3196.0,...,1.0,2.0,224.0,104.0,2000.0,Kingston,-38.03710,145.11717,1,2546.0
2525,38,127 George St,4,h,1780000.0,S,Peter,7/11/2016,1.6,3065.0,...,1.0,1.0,186.0,-1.0,1902.0,Yarra,-37.80480,144.98060,1,5825.0
11535,80,1/34 Longmuir Rd,2,u,655000.0,S,Darren,15/07/2017,14.5,3087.0,...,1.0,1.0,298.0,98.0,1975.0,Banyule,-37.71098,145.08926,1,2329.0
9832,152,3/13 Graham Rd,3,u,590000.0,S,Mitchell,24/06/2017,31.2,3197.0,...,1.0,1.0,179.0,-1.0,-1.0,Kingston,-38.07483,145.13133,1,1989.0
11080,254,40 Anzac Av,5,h,790000.0,S,Mason,12/08/2017,26.1,3099.0,...,2.0,2.0,1034.0,225.0,1985.0,Nillumbik,-37.63826,145.19910,7,1345.0


In [140]:
melbourne_data=melbourne_data.assign(Type=type_col)

In [141]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
11655,194,9a Ivan Av,2,2,724000.0,S,O'Brien,22/07/2017,27.0,3196.0,...,1.0,2.0,224.0,104.0,2000.0,Kingston,-38.03710,145.11717,1,2546.0
2525,38,127 George St,4,0,1780000.0,S,Peter,7/11/2016,1.6,3065.0,...,1.0,1.0,186.0,-1.0,1902.0,Yarra,-37.80480,144.98060,1,5825.0
11535,80,1/34 Longmuir Rd,2,2,655000.0,S,Darren,15/07/2017,14.5,3087.0,...,1.0,1.0,298.0,98.0,1975.0,Banyule,-37.71098,145.08926,1,2329.0
9832,152,3/13 Graham Rd,3,2,590000.0,S,Mitchell,24/06/2017,31.2,3197.0,...,1.0,1.0,179.0,-1.0,-1.0,Kingston,-38.07483,145.13133,1,1989.0
11080,254,40 Anzac Av,5,0,790000.0,S,Mason,12/08/2017,26.1,3099.0,...,2.0,2.0,1034.0,225.0,1985.0,Nillumbik,-37.63826,145.19910,7,1345.0


In [142]:
melbourne_data = melbourne_data.drop(columns=['Address','Method','SellerG','Date','Postcode','CouncilArea','Lattitude','Longtitude'])

In [143]:
melbourne_data.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
11655,194,2,2,724000.0,27.0,2.0,1.0,2.0,224.0,104.0,2000.0,1,2546.0
2525,38,4,0,1780000.0,1.6,4.0,1.0,1.0,186.0,-1.0,1902.0,1,5825.0
11535,80,2,2,655000.0,14.5,2.0,1.0,1.0,298.0,98.0,1975.0,1,2329.0
9832,152,3,2,590000.0,31.2,3.0,1.0,1.0,179.0,-1.0,-1.0,1,1989.0
11080,254,5,0,790000.0,26.1,5.0,2.0,2.0,1034.0,225.0,1985.0,7,1345.0


In [144]:
# target = melbourne_data.pop('Price')

In [145]:
train = melbourne_data.sample(frac=0.8,random_state=200)
target_train = train.pop('Price')
test = melbourne_data.drop(train.index)
target_test = test.pop('Price')


In [208]:
class MultiPerceptron(tf.keras.Model):

    def __init__(self):
        super(MultiPerceptron, self).__init__()
        self.dense1 = tf.keras.layers.Dense(12,activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(50,activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(50,activation=tf.nn.relu)
        self.dense4 = tf.keras.layers.Dense(1)
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        if training:
            x = self.dropout(x, training=training)
        y = self.dense2(x)
        y2 = self.dense3(y)
        return self.dense4(y2)

model = MultiPerceptron()
model.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy'])

In [209]:
dataset = tf.data.Dataset.from_tensor_slices((train.values, target_train.values))
dataset.__dict__

{'_tensors': [<tf.Tensor: shape=(10864, 12), dtype=float64, numpy=
  array([[ 5.0000e+00,  2.0000e+00,  0.0000e+00, ..., -1.0000e+00,
           1.0000e+00,  1.1918e+04],
         [ 9.6000e+01,  3.0000e+00,  0.0000e+00, ...,  1.9700e+03,
           1.0000e+00,  3.5890e+03],
         [ 1.1000e+01,  4.0000e+00,  0.0000e+00, ...,  1.9830e+03,
           1.0000e+00,  7.4850e+03],
         ...,
         [ 4.2000e+01,  3.0000e+00,  2.0000e+00, ...,  1.9950e+03,
           1.0000e+00,  8.8700e+03],
         [ 6.2000e+01,  3.0000e+00,  0.0000e+00, ...,  1.8800e+03,
           1.0000e+00,  6.7860e+03],
         [ 3.3000e+01,  2.0000e+00,  2.0000e+00, ..., -1.0000e+00,
           1.0000e+00,  1.3240e+04]])>,
  <tf.Tensor: shape=(10864,), dtype=float64, numpy=array([ 890000.,  565000.,  822000., ...,  880000., 1263000.,  530000.])>],
 '_structure': (TensorSpec(shape=(12,), dtype=tf.float64, name=None),
  TensorSpec(shape=(), dtype=tf.float64, name=None)),
 '_variant_tensor_attr': <tf.Tensor: shap

In [210]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 5.0000e+00  2.0000e+00  0.0000e+00  5.2000e+00  2.0000e+00  1.0000e+00
  1.0000e+00  2.5900e+02 -1.0000e+00 -1.0000e+00  1.0000e+00  1.1918e+04], Target: 890000.0
Features: [9.600e+01 3.000e+00 0.000e+00 1.080e+01 3.000e+00 2.000e+00 1.000e+00
 5.660e+02 7.800e+01 1.970e+03 1.000e+00 3.589e+03], Target: 565000.0
Features: [1.100e+01 4.000e+00 0.000e+00 9.900e+00 4.000e+00 2.000e+00 2.000e+00
 2.500e+02 1.940e+02 1.983e+03 1.000e+00 7.485e+03], Target: 822000.0
Features: [ 2.1000e+01  2.0000e+00  2.0000e+00  5.4000e+00  2.0000e+00  1.0000e+00
  1.0000e+00  8.4700e+02 -1.0000e+00 -1.0000e+00  1.0000e+00  1.0331e+04], Target: 600000.0
Features: [ 7.000e+00  4.000e+00  0.000e+00  1.300e+01  4.000e+00  1.000e+00
  4.000e+00  7.860e+02 -1.000e+00 -1.000e+00  1.000e+00  6.795e+03], Target: 1420000.0


In [211]:
train_dataset = dataset.shuffle(len(train)).batch(1)

In [212]:
test_dataset = tf.data.Dataset.from_tensor_slices((test.values, target_test.values))

In [213]:
test_dataset = test_dataset.batch(1)

In [214]:
model.fit(train_dataset, epochs=20)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

10864/10864 [==============================] - 9s 791us/step - loss: 868818747392.0000 - accuracy: 0.0000e+00
Epoch 2/20
10864/10864 [==============================] - 9s 839us/step - loss: 850260393984.0000 - accuracy: 0.0000e+00
Epoch 3/20
10864/10864 [==============================] - 9s 864us/step - loss: 761664307200.0000 - accuracy: 0.0000e+00
Epoch 4/20
10864/10864 [==============================] - 9s 871us/step - loss: 930695348224.0000 - accuracy: 0.0000e+00
Epoch 5/20
10864/10864 [

In [215]:
model.predict(test_dataset)

array([[502751.7 ],
       [300348.9 ],
       [597847.4 ],
       ...,
       [671357.8 ],
       [473741.5 ],
       [620768.06]], dtype=float32)

In [216]:
for x in test_dataset:
    print(x)

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[8.000e+01, 2.000e+00, 2.000e+00, 1.450e+01, 2.000e+00, 1.000e+00,
        1.000e+00, 2.980e+02, 9.800e+01, 1.975e+03, 1.000e+00, 2.329e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([655000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.520e+02,  3.000e+00,  2.000e+00,  3.120e+01,  3.000e+00,
         1.000e+00,  1.000e+00,  1.790e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  1.989e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([590000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[8.20e+01, 3.00e+00, 0.00e+00, 1.14e+01, 3.00e+00, 1.00e+00,
        1.00e+00, 6.25e+02, 1.35e+02, 1.96e+03, 1.00e+00, 3.54e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([945000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 2.000e+00,  4.000e+00,  0.000e+00,  1.340e+01,  4.000e+00,
         2.000e+00,  2.000e+00,  9.230e+02, -1.000e+00, -1.000e+00,
        

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[2.4000e+01, 3.0000e+00, 0.0000e+00, 9.2000e+00, 3.0000e+00,
        1.0000e+00, 1.0000e+00, 7.5300e+02, 1.1800e+02, 1.9400e+03,
        1.0000e+00, 1.0412e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1530000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.0000e+01,  3.0000e+00,  0.0000e+00,  4.6000e+00,  2.0000e+00,
         1.0000e+00,  4.0000e+00,  6.7700e+02, -1.0000e+00, -1.0000e+00,
         1.0000e+00,  1.1308e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([3120000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 6.600e+01,  3.000e+00,  0.000e+00,  9.200e+00,  2.000e+00,
         2.000e+00,  1.000e+00,  7.820e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  3.873e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([750000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[7.800e+01, 2.000e+00, 0.000e+00, 1.210e+01, 2.000e+00, 1.000e+00,
    

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[3.40e+01, 3.00e+00, 0.00e+00, 1.12e+01, 3.00e+00, 1.00e+00,
        1.00e+00, 5.84e+02, 1.53e+02, 1.96e+03, 1.00e+00, 8.87e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([922000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[1.430e+02, 3.000e+00, 0.000e+00, 1.400e+01, 3.000e+00, 1.000e+00,
        1.000e+00, 3.410e+02, 7.100e+01, 1.965e+03, 1.000e+00, 4.294e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([386000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 6.000e+01,  3.000e+00,  1.000e+00,  1.170e+01,  4.000e+00,
         2.000e+00,  2.000e+00,  2.170e+02, -1.000e+00,  1.993e+03,
         1.000e+00,  5.678e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([752000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[5.0000e+00, 4.0000e+00, 0.0000e+00, 5.2000e+00, 4.0000e+00,
        3.0000e+00, 1.0000e+00, 3.6500e+02, 2.0900e+02, 1.8900e+03,
        

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[4.000e+00, 2.000e+00, 2.000e+00, 1.350e+01, 2.000e+00, 1.000e+00,
        1.000e+00, 1.410e+02, 7.300e+01, 2.013e+03, 1.000e+00, 3.464e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([515000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 3.000e+00,  3.000e+00,  1.000e+00,  1.470e+01,  3.000e+00,
         1.000e+00,  2.000e+00,  2.520e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  3.692e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([800000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[6.600e+01, 3.000e+00, 0.000e+00, 9.200e+00, 3.000e+00, 1.000e+00,
        1.000e+00, 5.840e+02, 8.100e+01, 1.900e+03, 1.000e+00, 3.873e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([840000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 4.90e+01,  2.00e+00,  0.00e+00,  8.00e+00,  2.00e+00,  1.00e+00,
         2.00e+00,  1.83e+02, -1.00e+00, -1.00e+00,  1.00e

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 7.900e+01,  2.000e+00,  0.000e+00,  6.600e+00,  2.000e+00,
         1.000e+00,  1.000e+00,  1.160e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  2.417e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([785000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 7.400e+01,  3.000e+00,  1.000e+00,  1.600e+00,  3.000e+00,
         1.000e+00,  2.000e+00,  1.250e+02, -1.000e+00,  1.990e+03,
         1.000e+00,  4.553e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1308000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 7.800e+01,  4.000e+00,  0.000e+00,  1.210e+01,  4.000e+00,
         2.000e+00,  2.000e+00,  6.720e+02, -1.000e+00,  1.950e+03,
         1.000e+00,  2.606e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([815000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[8.300e+01, 4.000e+00, 0.000e+00, 1.020e+01, 4.000e+00, 3.000e+00,
        2.000e+00

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[8.000e+00, 3.000e+00, 0.000e+00, 1.120e+01, 3.000e+00, 2.000e+00,
        2.000e+00, 6.810e+02, 1.350e+02, 1.940e+03, 1.000e+00, 2.165e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([882500.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.410e+02,  5.000e+00,  0.000e+00,  1.670e+01,  5.000e+00,
         2.000e+00,  2.000e+00,  5.900e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  5.837e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1115000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[2.000e+01, 4.000e+00, 0.000e+00, 6.400e+00, 4.000e+00, 2.000e+00,
        1.000e+00, 1.540e+02, 1.610e+02, 2.007e+03, 1.000e+00, 7.570e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([977000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[8.000e+00, 3.000e+00, 0.000e+00, 1.200e+01, 3.000e+00, 1.000e+00,
        2.000e+00, 6.270e+02, 1.360e+02, 1.950e+03, 1.000

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[6.000e+00, 3.000e+00, 0.000e+00, 2.100e+00, 3.000e+00, 1.000e+00,
        1.000e+00, 1.810e+02, 1.170e+02, 1.880e+03, 1.000e+00, 5.943e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1405000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.700e+02,  3.000e+00,  0.000e+00,  1.210e+01,  3.000e+00,
         1.000e+00,  2.000e+00,  6.330e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  1.414e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([740000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 3.000e+00,  3.000e+00,  0.000e+00,  2.980e+01,  3.000e+00,
         1.000e+00,  2.000e+00,  5.920e+02, -1.000e+00, -1.000e+00,
         7.000e+00,  4.718e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([412250.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[5.0000e+00, 2.0000e+00, 0.0000e+00, 5.2000e+00, 2.0000e+00,
        1.0000e+00, 1.0000e+00, 6.1000e+01,

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.8000e+01,  2.0000e+00,  2.0000e+00,  7.8000e+00,  2.0000e+00,
         1.0000e+00,  1.0000e+00,  0.0000e+00, -1.0000e+00,  2.0070e+03,
         1.0000e+00,  1.1204e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([445000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 1.4800e+02,  3.0000e+00,  1.0000e+00,  2.4800e+01,  3.0000e+00,
         2.0000e+00,  2.0000e+00,  1.8690e+03,  5.0200e+02, -1.0000e+00,
         1.0000e+00,  1.0788e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1900000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[1.1000e+02, 4.0000e+00, 0.0000e+00, 3.1700e+01, 4.0000e+00,
        2.0000e+00, 1.0000e+00, 1.0070e+03, 1.1600e+02, 1.9800e+03,
        1.0000e+00, 1.4092e+04]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([600000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[6.000e+00, 2.000e+00, 0.000e+00, 3.800e+00, 2.000e+00, 1.00

(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 2.180e+02,  3.000e+00,  0.000e+00,  3.230e+01,  3.000e+00,
         2.000e+00,  2.000e+00,  6.490e+02, -1.000e+00, -1.000e+00,
         1.000e+00,  3.728e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([662500.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 8.40e+01,  4.00e+00,  0.00e+00,  5.10e+00,  4.00e+00,  1.00e+00,
         1.00e+00,  2.30e+02, -1.00e+00, -1.00e+00,  1.00e+00,  4.38e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1605000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[6.30e+01, 4.00e+00, 0.00e+00, 1.24e+01, 4.00e+00, 2.00e+00,
        1.00e+00, 6.62e+02, 2.40e+02, 1.95e+03, 1.00e+00, 5.42e+03]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1375000.])>)
(<tf.Tensor: shape=(1, 12), dtype=float64, numpy=
array([[ 2.000e+00,  4.000e+00,  0.000e+00,  8.100e+00,  4.000e+00,
         1.000e+00,  1.000e+00,  4.430e+02, -1.000e+00, -1.000e+00,
      